In [ ]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test
coll = db.restaurants

# B-tree indexes

In [ ]:
coll.drop_indexes()

In [ ]:
coll.find_one()

## Single-field index (range-based)

In [ ]:
coll.create_index('name')

In [ ]:
coll.index_information()

## Multi-field index (range-based)

In [ ]:
coll.drop_indexes()

In [ ]:
coll.create_index([
    ('borough', 1),
    ('cuisine', 1)])

In [ ]:
coll.index_information()

## Single-field index (hashed)

In [ ]:
coll.drop_indexes()

In [ ]:
coll.create_index([
    ('name', 'hashed')])


In [ ]:
coll.index_information()

## Unique Indexes

In [ ]:
coll.drop_indexes()

In [ ]:
coll.create_index('restaurant_id', unique=True)

In [ ]:
doc = coll.find_one()

In [ ]:
doc

In [ ]:
try:
    coll.insert_one({'restaurant_id': doc['restaurant_id']})
except Exception as err:
    print('Exception during insert\n{}'.format(err))

## Sparse Indexes

In [ ]:
coll.drop_indexes()

In [ ]:
coll.create_index('new_field')

In [ ]:
coll.index_information()

In [ ]:
coll.find({'new_field': {'$exists': True}}).explain()

In [ ]:
coll.count()

In [ ]:
coll.find({'new_field': {'$exists': True}}).count()

In [ ]:
coll.drop_indexes()
coll.create_index('new_field', sparse=True)

In [ ]:
coll.index_information()

In [ ]:
coll.find({'new_field': {'$exists': True}}).explain()

In [ ]:
coll.find({'new_field': {'$exists': True}}).count()

## TTL Indexes

In [ ]:
ttl = db['ttl']

In [ ]:
ttl.drop_indexes()

In [ ]:
ttl.create_index('ts', expireAfterSeconds=3)

In [ ]:
ttl.index_information()

In [ ]:
from datetime import datetime
import time

In [ ]:
doc = {'ts': datetime.utcnow()}
ttl.insert_one(doc)
for seconds in range(0, 60, 5):
    doc = ttl.find_one()
    print('@ {:2d}s, doc = {}'.format(seconds, doc))
    if doc is None:
        break
    time.sleep(5)

In [ ]:
coll.create_index?

# Geospatial indexes and search

In [ ]:
doc = coll.find_one()
doc

In [ ]:
curs = coll.find(
    {'address.coord': {'$nearSphere': doc['address']['coord']}})

In [ ]:
try:
    next(curs)
except Exception as err:
    print('Error advancing cursor:\n{}'.format(err))

In [ ]:
coll.drop_indexes()
coll.create_index([
    ('address.coord', '2d')
])

In [ ]:
curs = coll.find(
    {'address.coord': {'$nearSphere': doc['address']['coord']}},
    {'address': 1},
    limit=5)
for r in curs:
    print('Address: {address[building]} {address[street]} {address[zipcode]}'.format(
        address=r['address']))


In [ ]:
coll.drop_indexes()
coll.create_index([
    ('address.coord', '2dsphere')
])

In [ ]:
curs = coll.find(
    {'address.coord': {'$nearSphere': doc['address']['coord']}},
    {'address': 1},
    limit=5)
for r in curs:
    print('Address: {address[building]} {address[street]} {address[zipcode]}'.format(
        address=r['address']))


# Full-text Indexes and Search

In [ ]:
import re
q_re = coll.find({'name': re.compile('^Dunkin')})
q_re.count()

In [ ]:
coll.create_index([('name', 'text')])

In [ ]:
q_text = coll.find({'$text': {
    '$search': 'Dunkin',
    '$caseSensitive': True
}})
q_text.count()

In [ ]:
found_with_re = {doc['_id'] for doc in q_re}
found_with_text = {doc['_id'] for doc in q_text}

In [ ]:
print('Found with re but not text:')
for _id in found_with_re - found_with_text:
    print(coll.find_one({'_id': _id}, {'name': 1}))
print('Found with text but not re:')
for _id in found_with_text - found_with_re:
    print(coll.find_one({'_id': _id}, {'name': 1}))

In [ ]:
coll.drop_indexes()

In [ ]:
coll.create_index([("$**", "text")])

In [ ]:
q_text = coll.find({'$text': {
    '$search': 'queen'     # finds everything with queen in name, cuisine, borough, or address (incl. "Queens")
}})
q_text.count()